In [134]:
import numpy as np
from torch.nn import ConvTranspose2d
from torch import from_numpy
from numpy.testing import assert_array_equal

In [138]:
def assert_equal_own_and_torch(own, torch, input, weight, bias):
    torch.weight.data = from_numpy(weight).float()
    torch.bias.data = from_numpy(bias).float()
    own_result = np.floor(own)
    torch_result = np.floor(torch(from_numpy(input).float()).detach().numpy())
    #print(np.array(own_result).shape)
    #print(np.array(torch_result).shape)
    print("Мой результат")
    print(own_result)
    print("Результат библиотеки")
    print(torch_result)
    assert_array_equal(own_result, torch_result)

In [353]:
def conv2d(input, weight, bias, in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1,
                   bias_enabled=True, padding_mode='zeros',):
    out = []
   
    h_in, w_in = input.shape[1], input.shape[2]
    kernel_size = (kernel_size, kernel_size) if not isinstance(kernel_size, tuple) else kernel_size
    stride = (stride, stride) if not isinstance(stride, tuple) else stride
    dilation = (dilation, dilation) if not isinstance(dilation, tuple) else dilation

    padding = padding if isinstance(padding, tuple) else (padding, padding)

    h_out = int((h_in + 2 * padding[0] - dilation[0] * (kernel_size[0] - 1) - 1) / stride[0] + 1)
    w_out = int((w_in + 2 * padding[1] - dilation[1] * (kernel_size[1] - 1) - 1) / stride[1] + 1)

    #out.append(np.zeros((out_channels, h_out, w_out)))
    out = np.zeros((1, out_channels, h_out, w_out))
    for c_out in range(out_channels):
        for y_out in range(h_out):
            for x_out in range(w_out):
                summation = 0
                for c_in in range(in_channels):
                    for kernel_y in range(kernel_size[0]):
                        for kernel_x in range(kernel_size[1]):
                            y_in = y_out * stride[0] + kernel_y * dilation[0] - padding[0]
                            x_in = x_out * stride[1] + kernel_x * dilation[1] - padding[1]
                            if 0 <= y_in < h_in and 0 <= x_in < w_in:
                                summation += input[c_in, y_in, x_in] * weight_data[c_out, c_in, kernel_y, kernel_x]
                            elif padding_mode == 'replicate':
                                y_in = max(0, min(y_in, h_in - 1))
                                x_in = max(0, min(x_in, w_in - 1))
                                summation += input[c_in, y_in, x_in] * weight_data[c_out, c_in, kernel_y, kernel_x]
                out[0, c_out, y_out, x_out] = summation + (bias[c_out] if bias_enabled else 0)
                
    return np.array(out)

In [359]:
def conv_transpose2d(input, weight, bias, in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1,
                   bias_enabled=True, padding_mode='zeros'):
    
    batch_size, in_channels, h_in, w_in = input.shape
    _, out_channels, _, _ = weight.shape
    kernel_size = weight.shape[2]

    out_height = (h_in - 1) * stride - 2 * padding + kernel_size
    out_width = (w_in - 1) * stride - 2 * padding + kernel_size

    output = np.zeros((batch_size, out_channels, out_height, out_width))
    weight_flipped = np.flip(np.flip(weight, axis=-1), axis=-2)
    
    for b in range(batch_size):
        for c_out in range(out_channels):
            for c_in in range(in_channels):
                conv_result = conv2d(input[b, c_in, :, :].reshape(1,h_in,w_in), weight_flipped[b, c_out, :, :].reshape(1,kernel_size,kernel_size), bias, 1, 1, 
                                     kernel_size, stride, 1, dilation, bias_enabled, padding_mode)

                output[b, c_out, :, :] += conv_result.reshape(out_height,out_width)[::stride, ::stride] 
        
    return output

channel_1 = [[1, 5, 5, 6], [7, 11, 2, 11], [4, 8, 5, 3], [1, 2, 3, 4]]
channel_2 = [[1, 1, 1, 1], [2, 2, 2, 2], [3, 3, 3, 3], [4, 4, 4, 4]]
weight_1 = [[1, 1], [1, 1]]
weight_2 = [[1, 1], [1, 1]]
input_data = np.array([[channel_1, channel_2]])
weight_data = np.array([[weight_1, weight_2], [weight_1, weight_2]])
bias = np.array([0, 0])

result = conv_transpose2d(input_data, weight_data, bias, in_channels=1, out_channels=1, kernel_size=2)

torch_result = ConvTranspose2d(in_channels=2, out_channels=2, kernel_size=2, stride=1)
assert_equal_own_and_torch(result, torch_result, input_data, weight_data, bias)


Мой результат
[[[[ 2.  8. 12. 13.  7.]
   [11. 30. 29. 30. 20.]
   [16. 40. 36. 31. 19.]
   [12. 29. 32. 29. 14.]
   [ 5. 11. 13. 15.  8.]]

  [[ 2.  8. 12. 13.  7.]
   [11. 30. 29. 30. 20.]
   [16. 40. 36. 31. 19.]
   [12. 29. 32. 29. 14.]
   [ 5. 11. 13. 15.  8.]]]]
Результат библиотеки
[[[[ 2.  8. 12. 13.  7.]
   [11. 30. 29. 30. 20.]
   [16. 40. 36. 31. 19.]
   [12. 29. 32. 29. 14.]
   [ 5. 11. 13. 15.  8.]]

  [[ 2.  8. 12. 13.  7.]
   [11. 30. 29. 30. 20.]
   [16. 40. 36. 31. 19.]
   [12. 29. 32. 29. 14.]
   [ 5. 11. 13. 15.  8.]]]]
